In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import inspect

from sarsen import apps

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1_slc_iw_burst))

In [ ]:
product_urlpath = "data/S1B_IW_SLC__1SDV_20211223T051121_20211223T051148_030148_039993_BA4B.SAFE"
measurement_group = "IW3/VV/4"
dem_urlpath = "data/Rome-30m-DEM.tif"
output_urlpath = "GRD.tif"

swath_polarization_id, _, burst_index = measurement_group.rpartition("/")
orbit_group = f"{swath_polarization_id}/orbit"

orbit_group

In [ ]:
import xarray as xr

burst_sar = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group)  # type: ignore
burst_sar

In [ ]:
_ = abs(burst_sar.measurement).plot(vmax=350, figsize=(15, 4))

In [ ]:
orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group)  # type: ignore
orbit_ecef

In [ ]:
from sarsen import scene

dem_raster = scene.open_dem_raster(dem_urlpath)
dem_raster

In [ ]:
_ = dem_raster.plot(figsize=(12, 8))

In [ ]:
from sarsen import orbit

dem_3d = scene.make_dem_3d(dem_raster)
dem_ecef = scene.transform_dem_3d(dem_3d)

orbit_interpolator = orbit.OrbitPolyfitIterpolator.from_position(
    orbit_ecef.position
)
position_sar = orbit_interpolator.position(burst_sar.azimuth_time)
velocity_sar = orbit_interpolator.velocity(burst_sar.azimuth_time)

position_sar

In [ ]:
from sarsen import geocoding

dem_coords = geocoding.backward_geocode(
    dem_ecef, position_sar.azimuth_time, position_sar, velocity_sar
)
dem_coords

In [ ]:
geocoded = abs(burst_sar).sel(
    azimuth_time=dem_coords.azimuth_time,
    slant_range_time=dem_coords.slant_range_time,
    method="nearest",
)
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded

In [ ]:
_ = geocoded.measurement.plot(vmax=350, figsize=(12, 8))

In [ ]:
geocoded.rio.to_raster(
    output_urlpath,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="DEFLATE",
    num_threads="ALL_CPUS",
)